In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
url = 'https://www.hikingupward.com/'
data = requests.get(url)
data.status_code
# print(data.text)

soup = BeautifulSoup(data.text, 'html.parser')
# soup

In [3]:
##TODO: convert to function/class
url_links = [url + url_link.attrs.get('href')
             for nav in soup.find_all('div', {'class': 'navigation'})
                 for url_link in nav.find_all('a', href = True)]
print(len(url_links))
# url_links

407


Scrape these features:

hike_len_in_mi, difficulty_rating, streams_rating, views_rating, solitude_rating, camping_rating, 

hiking_time, elevation_gain

In [ ]:
## TODO: implement else statement for if there is more than one hike on a page
## TODO: scrape the geocoordinates of the parking lot

In [4]:
hike = []

def get_hike_data(hiking_upward_url: str) -> pd.DataFrame:
    

    hike_content = requests.get(hiking_upward_url)
    if hike_content.status_code == 200:
        try:
            hike_soup = BeautifulSoup(hike_content.text, 'html.parser')
            tables = hike_soup.find('table', {'class': 'hike_pages'})
            if len(tables.find_all('tr')) == 5:
                rating_table = tables.find_all('tr')[1].find_all('td')
                time_elev = tables.find_all('tr')[2].find_all('td')[1]

                try:
                    hike_len_in_mi = float(rating_table[0].text.replace("mls", "").strip())
                except: 
                    hike_len_in_mi = np.nan

                try:
                    difficulty_rating = int(rating_table[1].find('img').get('src').replace("../../images/stars/Star", "").replace("/images/stars/Star", "").replace("_clear.gif", "").replace("_grey.gif", "").replace("_red.gif", ""))
                except: 
                    difficulty_rating = np.nan

                try:
                    streams_rating = int(rating_table[2].find('img').get('src').replace("../../images/stars/Star", "").replace("/images/stars/Star", "").replace("_clear.gif", "").replace("_grey.gif", "").replace("_red.gif", ""))
                except: 
                    streams_rating = np.nan

                try:
                    views_rating = int(rating_table[3].find('img').get('src').replace("../../images/stars/Star", "").replace("/images/stars/Star", "").replace("_clear.gif", "").replace("_grey.gif", "").replace("_red.gif", "")) 
                except: 
                    views_rating = np.nan

                try: 
                    solitude_rating = int(rating_table[4].find('img').get('src').replace("../../images/stars/Star", "").replace("/images/stars/Star", "").replace("_clear.gif", "").replace("_grey.gif", "").replace("_red.gif", "")) 
                except: 
                    solitude_rating = np.nan

                try:
                    camping_rating = int(rating_table[5].find('img').get('src').replace("../../images/stars/Star", "").replace("/images/stars/Star", "").replace("_clear.gif", "").replace("_grey.gif", "").replace("_red.gif", "")) 
                except: 
                    camping_rating = np.nan
               
                 ##av.note: to be implemented later
#                 try:
#                     hiking_duration = str(time_elev.contents[0].strip())
#                 except: 
#                     hiking_duration = np.nan

#                 try:
#                     elevation_gain_ft = float(time_elev.contents[2].replace('ft', '').replace(',', '').replace('with three different ascents', '').replace('with multiple ascents', '').replace('with two ascents', '').replace('with two different ascents', '').strip())
#                 except: 
#                     elevation_gain_ft = np.nan

                df = pd.DataFrame({'hike_url': str(hiking_upward_url), 
                                   'hike_name': str(hike_soup.title.text),
                                   'park_abbreviation': hiking_upward_url.replace('https://www.hikingupward.com/', '').split('/', 1)[0],
                                   'hike_len_in_mi': hike_len_in_mi, 
                                   'difficulty_rating': difficulty_rating, 
                                   'streams_rating': streams_rating, 
                                   'views_rating': views_rating, 
                                   'solitude_rating': solitude_rating, 
                                   'camping_rating': camping_rating, 
#                                    'hiking_duration_str': hiking_duration, 
#                                    'elevation_gain_ft': elevation_gain_ft
                                  }, 
                                  index = [0]) 
                return df
        except Exception:
            pass

In [5]:
# test_link= url_links[0]
# print(test_link)
# test_df = get_hike_data(test_link)
# test_df

In [6]:
for url_link in tqdm(url_links):
    one_hike_df = get_hike_data(url_link)
    hike.append(one_hike_df)

100%|████████████████████████████████████████████████████████████████████████████████| 407/407 [01:38<00:00,  4.13it/s]


In [7]:
hike_df = pd.concat(hike, ignore_index = True)
print(hike_df.shape)

(382, 9)


## Data quality checks

In [8]:
hike_df.sample(10)

,hike_url,hike_name,park_abbreviation,hike_len_in_mi,difficulty_rating,streams_rating,views_rating,solitude_rating,camping_rating
114,https://www.hikingupward.com/JNF/HoopHole/,Hoop Hole Hike,JNF,9.7,4,2.0,1.0,3,3.0
119,https://www.hikingupward.com/JNF/MtRogersElkRi...,Mt Rogers/Elk Ridge Hike,JNF,12.1,5,4.0,5.0,3,4.0
206,https://www.hikingupward.com/SNP/BroadHollow/,Broad Hollow Hike,SNP,6.7,3,2.0,2.0,4,1.0
345,https://www.hikingupward.com/OVH/KeysGapBuzzar...,Keys Gap/Buzzard Rocks Hike,OVH,7.7,3,NaN,3.0,3,3.0
207,https://www.hikingupward.com/SNP/BrownMountain/,Brown Mountain Hike,SNP,10.3,4,3.0,4.0,3,4.0
324,https://www.hikingupward.com/OVH/BeaverdamRese...,Beaverdam Reservoir Hike,OVH,7.7,2,1.0,2.0,2,NaN
101,https://www.hikingupward.com/JNF/AngelsRestWoo...,Angels Rest - Woods Hole Hostel Hike,JNF,26.0,5,1.0,4.0,3,3.0
20,https://www.hikingupward.com/GWNF/EmeraldPond/,Emerald Pond Hike,GWNF,8.0,3,NaN,4.0,3,3.0
366,https://www.hikingupward.com/OVH/TrumboHollow/,Trumbo Hollow Hike,OVH,6.6,3,2.0,4.0,3,3.0
264,https://www.hikingupward.com/OMH/MarylandChall...,Maryland Challenge Hike,OMH,41.9,6,3.0,3.0,2,4.0


In [9]:
##av.notes:
## according to the sortable map:
## there are no hikes with difficulty 0
## streams, views,solitude, and camping should all have ratings from 0-6
hike_df.describe()

,hike_len_in_mi,difficulty_rating,streams_rating,views_rating,solitude_rating,camping_rating
count,382.000000,382.000000,285.000000,368.000000,382.000000,248.000000
mean,8.199215,3.198953,2.954386,3.434783,2.952880,2.971774
std,4.983624,1.257919,1.451574,1.460128,1.341007,1.423917
min,0.500000,1.000000,1.000000,0.000000,0.000000,1.000000
25%,5.000000,2.000000,2.000000,2.000000,2.000000,2.000000
50%,7.450000,3.000000,3.000000,4.000000,3.000000,3.000000
75%,10.600000,4.000000,4.000000,5.000000,4.000000,4.000000
max,41.900000,6.000000,6.000000,6.000000,6.000000,6.000000


In [10]:
# hike_df.info()

In [11]:
print(hike_df.park_abbreviation.isna().sum())
# print(hike_df.park_abbreviation.value_counts())
## percent of hikes in each park
hike_df.park_abbreviation.value_counts(normalize = True)*100

0


GWNF     25.130890
OVH      14.921466
SNP      12.827225
NCSP     11.256545
PNF       9.685864
JNF       6.806283
MNF       6.806283
OMH       3.926702
NNF       3.141361
OWVH      2.879581
GSMNP     1.308901
UNF       0.523560
OPH       0.523560
WMNF      0.261780
Name: park_abbreviation, dtype: float64

In [12]:
# hike_df[hike_df.hike_len_in_mi.isna()]

In [13]:
##av.note: to be implemented later
# hike_df[hike_df.elevation_gain_ft.isna()].sample(10)

In [14]:
hike_df[hike_df.streams_rating.isna()].sample(5)

,hike_url,hike_name,park_abbreviation,hike_len_in_mi,difficulty_rating,streams_rating,views_rating,solitude_rating,camping_rating
305,https://www.hikingupward.com/NCSP/RichlandBalsam/,Richland Balsam Hike,NCSP,1.4,2,NaN,2.0,2,NaN
132,https://www.hikingupward.com/MNF/ChimneyTop/,Chimney Top Hike,MNF,5.4,4,NaN,6.0,3,3.0
345,https://www.hikingupward.com/OVH/KeysGapBuzzar...,Keys Gap/Buzzard Rocks Hike,OVH,7.7,3,NaN,3.0,3,3.0
299,https://www.hikingupward.com/NCSP/MtJefferson/,Mt. Jefferson Hike,NCSP,4.3,2,NaN,4.0,2,NaN
37,https://www.hikingupward.com/GWNF/IngalsOverlook/,Ingals Overlook Hike,GWNF,2.4,1,NaN,3.0,4,NaN


In [15]:
hike_df[hike_df.camping_rating.isna()].sample(5)

,hike_url,hike_name,park_abbreviation,hike_len_in_mi,difficulty_rating,streams_rating,views_rating,solitude_rating,camping_rating
362,https://www.hikingupward.com/OVH/ShenandoahRiver/,Shenandoah River State Park Hike,OVH,8.5,2,5.0,3.0,3,NaN
333,https://www.hikingupward.com/OVH/FortunesCove/,Fortune's Cove Hike,OVH,5.3,4,2.0,4.0,4,NaN
353,https://www.hikingupward.com/OVH/PleasantGrove/,Pleasant Grove Hike,OVH,6.7,2,4.0,2.0,3,NaN
64,https://www.hikingupward.com/GWNF/OtterCreek/,Otter Creek Hike,GWNF,7.4,3,5.0,2.0,3,NaN
256,https://www.hikingupward.com/OMH/AntietamBurns...,Antietam Burnside Bridge Hike,OMH,4.4,2,5.0,2.0,1,NaN


In [16]:
hike_df[hike_df.views_rating.isna()].sample(5)

,hike_url,hike_name,park_abbreviation,hike_len_in_mi,difficulty_rating,streams_rating,views_rating,solitude_rating,camping_rating
182,https://www.hikingupward.com/PNF/LittleFallBra...,Little Fall Branch Falls Hike,PNF,0.6,1,3.0,NaN,4,NaN
294,https://www.hikingupward.com/NCSP/LowerCascade...,Lower Cascade Falls Hike,NCSP,0.9,1,4.0,NaN,3,NaN
371,https://www.hikingupward.com/OWVH/BeartownStat...,Beartown State Park Hike,OWVH,0.5,1,NaN,NaN,1,NaN
172,https://www.hikingupward.com/PNF/CoveCreekFalls/,Cove Creek Falls Hike,PNF,2.5,2,3.0,NaN,1,NaN
199,https://www.hikingupward.com/PNF/TomsCreekFalls/,Toms Creek Falls Hike,PNF,0.8,1,4.0,NaN,2,NaN


In [17]:
## these are probably ones that have two hikes!

In [18]:
##av.note: to be implemented later
## TODO: figure out how to scrape these (something to do with the font attribute, maybe?)
# print(hike_df[hike_df.elevation_gain_ft.isna() & hike_df.hiking_duration_str.isna()].shape)
# hike_df[hike_df.elevation_gain_ft.isna() & hike_df.hiking_duration_str.isna()]

## Data cleaning

In [19]:
## will need to replace NaN for Brandywine Recreation Area Hike 
hike_df.streams_rating.fillna(0, inplace = True)
hike_df.camping_rating.fillna(0, inplace = True)
hike_df.views_rating.fillna(0, inplace = True)

In [20]:
##av.note: to be implemented later
## get the state based on park abbreviation
# hike_df['state'] = 'TBD'

In [21]:
hike_df.describe()

,hike_len_in_mi,difficulty_rating,streams_rating,views_rating,solitude_rating,camping_rating
count,382.000000,382.000000,382.000000,382.000000,382.000000,382.000000
mean,8.199215,3.198953,2.204188,3.308901,2.952880,1.929319
std,4.983624,1.257919,1.796814,1.572024,1.341007,1.825089
min,0.500000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,5.000000,2.000000,0.000000,2.000000,2.000000,0.000000
50%,7.450000,3.000000,2.000000,3.000000,3.000000,2.000000
75%,10.600000,4.000000,4.000000,5.000000,4.000000,3.000000
max,41.900000,6.000000,6.000000,6.000000,6.000000,6.000000


## Write Data

In [23]:
hike_df.to_csv("../data/hiking_upward_data.csv", index = False)

### exploration that made the above function possible

In [24]:
hike_data = requests.get(url_links[0])
hike_soup = BeautifulSoup(hike_data.text, 'html.parser')
# print(hike_soup)

In [25]:
hike_soup.title.text

'Anthill/Slate Lick Hike'

In [26]:
## exploration

# for tables in hike_soup.find_all('table', {'class': 'hike_pages'}):
#     for table in tables.find_all('tr'):
#         print(table)
#         print("\n")
# #         for row in table.find_all('td'):
# #             print(row)
# #             print("\n")

In [27]:
tables = hike_soup.find('table', {'class': 'hike_pages'})
print(len(tables.find_all('tr')))
tables

5


<table border="1" cellpadding="4" cellspacing="0" class="hike_pages" width="100%">
<tr>
<td align="center" width="16%"><b><font color="#587127">Length</font></b></td>
<td align="center" width="16%"><b><font color="#587127">Difficulty</font></b></td>
<td align="center" width="16%"><b><font color="#587127">Streams</font></b></td>
<td align="center" width="16%"><b><font color="#587127">Views</font></b></td>
<td align="center" width="16%"><b><font color="#587127">Solitude</font></b></td>
<td align="center" width="16%"><b><font color="#587127">Camping</font></b></td>
</tr>
<tr>
<td align="center" bgcolor="#f6fceb"><b>11.5 mls</b></td>
<td align="center" bgcolor="#f6fceb"><img height="17" src="../../images/stars/Star4_clear.gif" width="18"/></td>
<td align="center" bgcolor="#f6fceb"><img height="17" src="../../images/stars/Star3_clear.gif" width="18"/></td>
<td align="center" bgcolor="#f6fceb"><img height="17" src="../../images/stars/Star2_clear.gif" width="18"/></td>
<td align="center" bgco

In [28]:
 tables.find_all('tr')[0].find_all('td')

[<td align="center" width="16%"><b><font color="#587127">Length</font></b></td>,
 <td align="center" width="16%"><b><font color="#587127">Difficulty</font></b></td>,
 <td align="center" width="16%"><b><font color="#587127">Streams</font></b></td>,
 <td align="center" width="16%"><b><font color="#587127">Views</font></b></td>,
 <td align="center" width="16%"><b><font color="#587127">Solitude</font></b></td>,
 <td align="center" width="16%"><b><font color="#587127">Camping</font></b></td>]

In [29]:
rating_table = tables.find_all('tr')[1].find_all('td')
rating_table

[<td align="center" bgcolor="#f6fceb"><b>11.5 mls</b></td>,
 <td align="center" bgcolor="#f6fceb"><img height="17" src="../../images/stars/Star4_clear.gif" width="18"/></td>,
 <td align="center" bgcolor="#f6fceb"><img height="17" src="../../images/stars/Star3_clear.gif" width="18"/></td>,
 <td align="center" bgcolor="#f6fceb"><img height="17" src="../../images/stars/Star2_clear.gif" width="18"/></td>,
 <td align="center" bgcolor="#f6fceb"><img height="17" src="../../images/stars/Star4_clear.gif" width="18"/></td>,
 <td align="center" bgcolor="#f6fceb"><img height="17" src="../../images/stars/Star3_clear.gif" width="18"/></td>]

In [30]:
hike_len_in_mi = float(rating_table[0].text.replace(" mls", ""))
print(hike_len_in_mi) # str - convert to float
difficulty_rating = int(rating_table[1].find('img').get('src').replace("../../images/stars/Star", "").replace("_clear.gif", "")) 
print(difficulty_rating) #str - convert to int
streams_rating = int(rating_table[2].find('img').get('src').replace("../../images/stars/Star", "").replace("_clear.gif", "")) 
print(streams_rating) #str - convert to int
views_rating = int(rating_table[3].find('img').get('src').replace("../../images/stars/Star", "").replace("_clear.gif", "")) 
print(views_rating) #str - convert to int
solitude_rating = int(rating_table[4].find('img').get('src').replace("../../images/stars/Star", "").replace("_clear.gif", "")) 
print(solitude_rating) #str - convert to int
camping_rating = int(rating_table[5].find('img').get('src').replace("../../images/stars/Star", "").replace("_clear.gif", "")) 
print(camping_rating) #str - convert to int

11.5
4
3
2
4
3


In [31]:
time_elev = tables.find_all('tr')[2].find_all('td')[1]
# print(time_elev)
hiking_duration = time_elev.contents[0]
print(hiking_duration) #str
elevation_gain_ft = time_elev.contents[2].replace(' ft', '').replace(',', '')
print(elevation_gain_ft) #str - convert to int/float

5.5 hours including breaks

                      2000
